Problem statement

$$min\, -8.1x_1-10.8x_2 $$
$$s.t. $$
$$ 0.8x_1 + 0.44x_2 \le 24000 +\theta_1 $$
$$ 0.05x_1 +0.1x_2\le2000 +\theta_2 $$
$$ 0.1x_1 + 0.36x_2 \le 6000 $$ 
$$ x_1, x_2 \ge 0 $$
$$ 0 \le \theta_1 \le 6000 $$
$$ 0 \le \theta_2 \le 500 $$


In [108]:
import pyomo.environ as pmo
import numpy as np

class GenericSolver:
    
    def __init__(self, A, b, m):
        # get no of var and constraints
        self.x_card = np.shape(A)[1]
        self.c_card = np.shape(A)[0]
        
        # transform A from matrix to dict
        A_init = {}
        for i in range(self.c_card):
            for j in range(self.x_card):
                A_init[(i+1, j+1)] = A[i, j]
        
        # transform b from vector to dict
        b_init = {}
        for i in range(self.c_card):
            b_init[i+1] = b[i]
            
        # transform m from vector to dict
        m_init = {}
        for i in range(self.x_card):
            m_init[i+1] = m[i]
        
        # define pyomo model
        self.model = pmo.ConcreteModel()
        self.model.n = pmo.RangeSet(1, self.x_card)
        self.model.c = pmo.RangeSet(1, self.c_card)
        self.model.A = pmo.Param(self.model.c, self.model.n, initialize=A_init)
        self.model.b = pmo.Param(self.model.c, initialize=b_init)
        self.model.m = pmo.Param(self.model.n, initialize=m_init)
        self.model.x = pmo.Var(self.model.n, domain=pmo.NonNegativeReals)
        self.model.dual = pmo.Suffix(direction=pmo.Suffix.IMPORT)
        self.model.constraints = pmo.ConstraintList()
        for c in self.model.c:
            self.model.constraints.add(
                sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n) <= self.model.b[c]
            )
        self.model.obj = pmo.Objective(
            expr=sum(self.model.m[i] * self.model.x[i] for i in self.model.n)
        )
        
        # define solver
        self.solverpath = 'C:\\w64\\glpsol'
        self.solver = pmo.SolverFactory('glpk', executable=self.solverpath)
    
        # define empty output entities
        self.soln = None
        self.duals = None
    
    def solve(self):
        self.solver.solve(self.model, tee=False)
        self.soln = np.empty([self.x_card])
        for i in range(self.x_card):
            self.soln[i] = self.model.x[i+1].value
        self.duals = np.empty([self.c_card])
        for c in range(self.c_card):
            self.duals[c] = -self.model.dual[self.model.constraints[c+1]]